# Setup

In [2]:
import gc
import numpy as np
import os
import sklearn.metrics
import tensorflow as tf

import models
import util

SEED = 5667
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Stay in top-level directory for consistency
if '\\src' in os.getcwd():
    os.chdir('..')

os.getcwd()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


'C:\\Users\\bradl\\Projects\\fair-child'

In [3]:
# Load data
xtrain, ytrain, xtest, ytest, xval, yval = util.load_preg_data_final(datafile='preterm')
# Convert class labels to binary labels
ytrain = util.outcome_to_binary(ytrain, outcome='preterm')  # Choose between early stillbirth, late stillbirth, and preterm
ytest = util.outcome_to_binary(ytest, outcome='preterm')
yval = util.outcome_to_binary(yval, outcome='preterm')

# Garbage collect just in case
gc.collect()

Loading train data for preterm...
	done.
Loading test data for preterm...
	done.
Loading val data for preterm...
	done.


0

# Logistic Regression

## Train

In [16]:
lr_late = models.build_logreg()
models.fit_logreg(lr_late, xtrain, ytrain)
models.save_pickle(lr_late, 'models/lr_late')

c:\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
# Example load and evaluate
# del lr_early
# lr_early = models.load_pickle('models/lr_early')
# print(lr_early.score(xtest, ytest))
# print(sklearn.metrics.classification_report(ytest, lr_early.predict(xtest), digits=4))

# Gradient Boosting

In [5]:
gb_late = models.build_gbdt()
models.fit_gbdt(gb_late, xtrain, ytrain, xval, yval)
models.save_pickle(gb_late, 'models/gb_late')

[100]	valid_0's auc: 0.839856	valid_0's binary_logloss: 0.0369084
[200]	valid_0's auc: 0.841067	valid_0's binary_logloss: 0.0639765
[300]	valid_0's auc: 0.841748	valid_0's binary_logloss: 0.0895245
[400]	valid_0's auc: 0.842002	valid_0's binary_logloss: 0.113431
[500]	valid_0's auc: 0.841903	valid_0's binary_logloss: 0.135838
[600]	valid_0's auc: 0.842038	valid_0's binary_logloss: 0.156736
[700]	valid_0's auc: 0.841989	valid_0's binary_logloss: 0.176169
[800]	valid_0's auc: 0.842022	valid_0's binary_logloss: 0.194216
[900]	valid_0's auc: 0.842154	valid_0's binary_logloss: 0.210931
[1000]	valid_0's auc: 0.842218	valid_0's binary_logloss: 0.226401
[1100]	valid_0's auc: 0.842233	valid_0's binary_logloss: 0.240696
[1200]	valid_0's auc: 0.842311	valid_0's binary_logloss: 0.253867
[1300]	valid_0's auc: 0.842373	valid_0's binary_logloss: 0.265998
[1400]	valid_0's auc: 0.842437	valid_0's binary_logloss: 0.277155
[1500]	valid_0's auc: 0.842479	valid_0's binary_logloss: 0.287402
[1600]	valid_0's

# Neural nets

In [ ]:
from importlib import reload
reload(models)

lrelu_late = models.build_NN_lrelu(input_len=xtrain.shape[1])  # Assuming xtrain is (batch_size, n_features)
lrelu_late.summary()

models.fit_NN_lrelu(lrelu_late, xtrain, ytrain, xval, yval)
models.save_NN(lrelu_late, 'models/lrelu_preterm')

Model: "NN_lrelu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
lrelu1 (Dense)               (None, 70)                2450      
_________________________________________________________________
dropout (Dropout)            (None, 70)                0         
_________________________________________________________________
lrelu2 (Dense)               (None, 80)                5680      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
output (Dense)               (None, 1)                 81        
Total params: 8,211
Trainable params: 8,211
Non-trainable params: 0
________________________________________________________

In [7]:
# del lrelu_early
# lrelu_early = models.load_NN('models/lrelu_early')
# lrelu_early.evaluate(xtest, ytest)  # Output is [loss, accuracy, auc]

In [8]:
selu_late = models.build_NN_selu(input_len=xtrain.shape[1])  # Assuming xtrain is (batch_size, n_features)
selu_late.summary()

models.fit_NN_selu(selu_late, xtrain, ytrain, xval, yval)
models.save_NN(selu_late, 'models/selu_late')

Model: "NN_selu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 27)]              0         
_________________________________________________________________
selu1 (Dense)                (None, 27)                756       
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 27)                0         
_________________________________________________________________
selu2 (Dense)                (None, 27)                756       
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 27)                0         
_________________________________________________________________
selu3 (Dense)                (None, 27)                756       
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (None, 27)                0   

INFO:tensorflow:Assets written to: models/selu_late\assets


In [9]:
# del selu_early
# selu_early = models.load_NN('models/selu_early')
# selu_early.evaluate(xtest, ytest)  # Output is [loss, accuracy, auc]